In [1]:
import pandas as pd

In [2]:
bu = pd.read_csv("yelp_business.csv")

In [3]:
# Removes all values that are not in Santa Barbara
bu = bu[bu["city"] == "Santa Barbara"]

In [4]:
# Removes all values that are not restaurants
bu["categories"] = bu["categories"].astype("str")
bu["categories"] = bu["categories"].apply(lambda x: set(x.split(", ")))

word_list = ["Restaurants", "Food", "Restaurant"]
bu["categories"] = bu["categories"].apply(lambda x: x.intersection(word_list))
bu = bu[bu["categories"] != set()]

In [5]:
# Removes all values that are not open
bu = bu[bu["is_open"] == 1]

In [6]:
# Drops unnessescary columns and does some formatting 
bu = bu.drop(columns=['_id', 'city', 'is_open', 'state', 'categories'])
bu["attributes.RestaurantsTakeOut"].fillna("False", inplace = True)
bu["attributes.RestaurantsTakeOut"] = bu["attributes.RestaurantsTakeOut"].replace("None", "False")
bu = bu.rename(columns={"attributes.RestaurantsTakeOut":"take_out"})

In [7]:
# Reads the reviews csv
re = pd.read_csv("yelp_review.csv")

In [8]:
# Make the st series which is indexed by tuples of business id and star value eg ("thisisanid", 1)
re = pd.merge(bu, re, how ='inner', on =['business_id']).drop(columns=["address", "take_out", "latitude", "longitude", "name", 
                                                               "review_count", "stars_x"]).rename(columns={"stars_y":"stars"})
st = re.groupby(["business_id", "stars"]).count()['review_id']

In [31]:
# Iterates through st and makes lists of star values, and adds them to the dictionary with the business id as the code
# stars["thisisanid"] = [0,0,0,10,50] is a for a business with id "thisisanid" that got 0 1,2, and 3 star reviews, 10 4 star and 50 5 star
stars = {}
for key in st.index:
    if key[0] not in stars:
        stars[key[0]] = [0,0,0,0,0]

    stars[key[0]][key[1]-1] = st[key]

In [33]:
star_list = list(stars.values())

for i in range(len(star_list)):
    star_sum = 0
    for star_val in range(len(star_list[i])):
        star_sum += (star_val + 1) * star_list[i][star_val]
        
    star_list[i] = star_sum/sum(star_list[i])

In [17]:
# sorts bu
bu = bu.sort_values(by='business_id')

In [35]:
# adds star/date lists to bu
bu['stars'] = star_list

In [39]:
# writes bu to the csv
bu.to_csv('yelp_business_clean.csv', index=False)

In [36]:
bu.head()

,address,take_out,business_id,latitude,longitude,name,review_count,stars
9418,1012 State St,True,-3AooxIkg38UyUdlz5oXdw,34.421931,-119.702185,Chase Restaurant,436,3.144144
76606,2036 Cliff Dr,True,-6jvfSJGprbfBD2QrS9zQw,34.402538,-119.724894,Mesa Produce,19,5.000000
62010,1417 San Andres St,True,-ALqLSTzkGDMscHdxA1NgA,34.418221,-119.715795,Su Casa Fresh Mexican Grill,29,4.344828
134706,1905 Cliff Dr,True,-BdYhP-12elmFV7oB1iv4A,34.401382,-119.722472,Corner Tap Room,41,4.795455
92579,31 W Carrillo St,True,-FM4CxOg4XXmX_Ebky_SiQ,34.420361,-119.702475,Finch & Fork,1405,4.204324
